# Get Sentinel-1 stacks for windthrown areas

The initial extended evoland chips were gotten from AWS Element84 S2 COGs. This dataset is missing a bunch of recently reprocessed sentinel 2 data from the beginning of the data availability.

This notebook aims to fill in any gaps using data from the official CDSE repository. 

To do this we follow a few steps. 

1. Get all dates of the chips available already
2. Cross-reference with CDSE stac catalog
3. Use SH Processing API to query missing chips

To find the best way to do this we need to find out approximate numbers of how many chips are missing.

In [1]:
from pathlib import Path
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
import traceback

from tqdm import tqdm
import rasterio
import geopandas as gpd
from sentinelhub import (
    geo_utils,
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    SHConfig,
    SentinelHubCatalog,
    SentinelHubDownloadClient,
)
from affine import Affine
import numpy as np

/home/jonas/Documents/Projects/2025/DISFOR/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [75]:
samples = gpd.read_parquet("../data/samples.parquet")

In [76]:
samples = samples.query("dataset==3")

In [9]:
config = SHConfig("download")

In [65]:
evalscript_body = """
AVAILABLE = new Set();
NOT_AVAILABLE = new Set();
TO_INCLUDE_SET = new Set(TO_INCLUDE);

function setup() {
    return {
        input: [{
            bands: ["VV", "VH"],
        }],
        output: TO_INCLUDE.map(date => ({
            id: date,
            bands: 2,
            sampleType: "FLOAT32"
        })),
        mosaicking: "ORBIT"
    };
}

function preProcessScenes(collections) {
    collections.scenes.orbits = collections.scenes.orbits.filter(function(orbit) {
        var orbitDateFrom = orbit.dateFrom.split("T")[0];
        var toGet = TO_INCLUDE_SET.has(orbitDateFrom);
        if(toGet){
          AVAILABLE.add(orbitDateFrom)
        }
        return toGet
    });
    return collections;
}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata){
  outputMetadata.userData = Array.from(AVAILABLE)
}

function evaluatePixel(samples) {
    let dataOutputs = {};
    let sampleIndex = 0;
    
    for(let date of TO_INCLUDE){
        if(AVAILABLE.has(date) && sampleIndex < samples.length){
            let sample = samples[sampleIndex];
            dataOutputs[date] = [
                Math.max(0, Math.log(sample.VV) * 0.21714724095 + 1), 
                Math.max(0, Math.log(sample.VH) * 0.21714724095 + 1),
            ];
            sampleIndex++;
        } else {
            dataOutputs[date] = [0, 0];
        }
    }
    return dataOutputs;
}
"""

In [66]:
def process_sample(sample, evalscript_body, config):
    """Process a single sample - designed to be run in parallel."""
    centroid = sample.geometry
    crs = CRS.get_utm_from_wgs84(centroid.x, centroid.y)
    x, y = geo_utils.transform_point((centroid.x, centroid.y), CRS.WGS84, crs)
    x_round = math.floor(x / 10) * 10
    y_round = math.ceil(y / 10) * 10
    resolution = 10
    size = 32
    bounds = (
        x_round - (size * resolution) // 2,
        y_round - (size * resolution) // 2,
        x_round + (size * resolution) // 2,
        y_round + (size * resolution) // 2,
    )

    bbox = BBox(bounds, crs=crs)
    time_interval = "2015-01-01", "2025-01-01"

    catalog = SentinelHubCatalog(config=config)
    search_iterator = catalog.search(
        DataCollection.SENTINEL1_IW,
        bbox=bbox,
        time=time_interval,
        fields={
            "include": ["id", "properties.datetime"],
            "exclude": [],
        },
    )

    results = list(search_iterator)
    available_timestamps = list(
        set(item["properties"]["datetime"][0:10] for item in results)
    )

    available_timestamps.sort()

    list_of_requests = []
    for year in range(15, 25):
        year_ts = [ts for ts in available_timestamps if ts[2:4] == str(year)]
        evalscript = (
            "//VERSION 3\n" + "TO_INCLUDE = " + json.dumps(year_ts) + evalscript_body
        )

        request_all_bands = SentinelHubRequest(
            evalscript=evalscript,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL1_IW.define_from(
                        "s1", service_url=config.sh_base_url
                    ),
                    time_interval=(f"20{year}-01-01", f"20{year + 1}-01-01"),
                    mosaicking_order=MosaickingOrder.LEAST_RECENT,
                    other_args={
                        "processing":
                        {
                            "backCoeff": "GAMMA0_TERRAIN",
                            "orthorectify": True,
                            "demInstance": "COPERNICUS_30"
                        }
                    }
                )
            ],
            responses=[
                SentinelHubRequest.output_response(output, MimeType.TIFF)
                for output in year_ts
            ]
            + [SentinelHubRequest.output_response("userdata", MimeType.JSON)],
            bbox=bbox,
            size=(size, size),
            config=config,
        )
        list_of_requests.append(request_all_bands)

    download_list = [request.download_list[0] for request in list_of_requests]
    # download data with multiple threads
    data = SentinelHubDownloadClient(config=config).download(
        download_list, max_threads=5
    )

    out_path = Path(f"./s1/{sample.sample_id}")
    out_path.mkdir(parents=True, exist_ok=True)
    for all_bands_response in data:
        to_save = list(all_bands_response["userdata.json"].values())
        profile = {
            "driver": "GTiff",
            "dtype": "float32",
            "width": 32,
            "height": 32,
            "count": 2,
            "crs": crs.epsg,
            "transform": Affine(10.0, 0.0, bounds[0], 0.0, -10.0, bounds[3]),
            "blockxsize": 512,
            "blockysize": 512,
            "tiled": True,
            "compress": "zstd",
            "interleave": "pixel",
        }

        for date in to_save:
            filename = f"{date}.tif"
            with rasterio.open(out_path / filename, "w", **profile) as dst:
                dst.write(np.transpose(all_bands_response[filename], (2, 0, 1)))

    return sample.sample_id, len(to_save), None

In [77]:
to_get = []
for sample_id in samples.sample_id:
    if not Path(f"./s1/{sample_id}").exists():
        to_get.append(sample_id)

In [78]:
to_get

[3529,
 3531,
 3532,
 3533,
 3534,
 3535,
 3536,
 3537,
 3538,
 3539,
 3540,
 3541,
 3542,
 3543,
 3544,
 3545,
 3546,
 3547,
 3548,
 3549,
 3550,
 3551,
 3552,
 3553,
 3554,
 3555,
 3556,
 3557,
 3558,
 3559,
 3560,
 3561,
 3562,
 3563,
 3564,
 3565,
 3566,
 3567,
 3568,
 3569,
 3570,
 3571,
 3572,
 3573,
 3574,
 3575,
 3576,
 3577,
 3578,
 3579,
 3580,
 3581,
 3582,
 3583,
 3584,
 3585,
 3586,
 3587,
 3588,
 3589,
 3590,
 3591,
 3592,
 3593,
 3594,
 3595,
 3596,
 3597,
 3598,
 3599,
 3600,
 3601,
 3602,
 3603,
 3604,
 3605,
 3606,
 3607,
 3608,
 3609,
 3610,
 3611,
 3612,
 3613,
 3614,
 3615,
 3616,
 3617,
 3618,
 3619,
 3620,
 3621,
 3622,
 3623,
 3624,
 3625,
 3626,
 3627,
 3628,
 3629,
 3630,
 3631,
 3632,
 3633,
 3634,
 3635,
 3636,
 3637,
 3638,
 3639,
 3640,
 3641,
 3642,
 3643,
 3644,
 3645,
 3646,
 3647,
 3648,
 3649,
 3650,
 3651,
 3652,
 3653,
 3654,
 3655,
 3656,
 3657,
 3658,
 3659,
 3660,
 3661,
 3662,
 3663,
 3664,
 3665,
 3666,
 3667,
 3668,
 3669,
 3670,
 3671,
 3672,

In [79]:
samples = samples.query("sample_id in @to_get")

In [80]:
# Main execution with ThreadPoolExecutor
max_workers = 4  # Adjust based on your system and API rate limits

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {
        executor.submit(
            process_sample, sample, evalscript_body, config
        ): sample.sample_id
        for _, sample in samples.iterrows()
    }

    for future in tqdm(as_completed(futures)):
        sample_id = futures[future]
        try:
            result_id, count, error = future.result()
            if error:
                print(f"Sample {sample_id} failed: {error}")
            else:
                print(f"Sample {sample_id} succeeded: {count} files written")
        except Exception:
            # Now this will catch real exceptions with full stack traces
            print(f"Sample {sample_id} failed with exception:")
            traceback.print_exc()

1it [01:51, 111.89s/it]

Sample 3532 succeeded: 91 files written


2it [01:56, 48.57s/it] 

Sample 3529 succeeded: 121 files written


3it [01:58, 27.64s/it]

...

Sample 3820 succeeded: 89 files written


292it [2:17:19, 29.33s/it]

Sample 3822 succeeded: 89 files written


293it [2:17:20, 28.13s/it]

Sample 3821 succeeded: 89 files written
